 # Todas las cuestiones relativas al preprocesado figuran en este Collab

#### **Recordatorio**: aunque todo este preprocesado se haya hecho en grupo, se recuerda que la redacción ha de ser personalizada.

---
---

# Length Features

Aquí se recogen algunas variables relacionadas con la longitud de la frase tanto a nivel de token como a nivel de caracter.

In [97]:
import pandas as pd
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/haha_2021_train.csv", sep=',')

In [98]:
train_dataframe.columns

Index(['id', 'text', 'is_humor', 'votes_no', 'votes_1', 'votes_2', 'votes_3',
       'votes_4', 'votes_5', 'humor_rating', 'humor_mechanism',
       'humor_target'],
      dtype='object')

In [99]:
text = train_dataframe[['id', 'text', 'is_humor', 'votes_no', 'votes_1', 'votes_2', 'votes_3',
       'votes_4', 'votes_5', 'humor_rating', 'humor_mechanism',
       'humor_target']]
text = text.iloc[4650:4660,:] # comentar esta

In [100]:
def len_token(tweet):
  tweet_tokens = tweet.replace('\n', ' ').split(' ')
  return len(tweet_tokens)
def n_characters_tweet(tweet):
  tweet = tweet.replace('\n', ' ')
  return len(tweet)

def get_len_token(text):
  return list(map(len_token, text['text'].to_list()))

def get_n_characters_tweet(text):
  return list(map(n_characters_tweet, text['text'].to_list()))

def get_n_characters_tweet(text):
  return list(map(n_characters_tweet, text['text'].to_list()))

text['len_token'] = get_len_token(text)

text['n_char'] = get_n_characters_tweet(text)

text['ratio'] = text['len_token']/text['n_char']

text

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target,len_token,n_char,ratio
4650,tweet4651,“Te voy a dejar encerrado todo el día y sin In...,1,0,2,0,2,1,0,2.40,NaN,NaN,30,139,0.215827
4651,tweet4652,Es verdad eso que dicen que después de los qui...,0,3,0,0,0,0,0,NaN,NaN,NaN,13,73,0.178082
4652,tweet4653,En mi defensa diré que fui educado mientras El...,0,3,0,0,0,0,0,NaN,NaN,NaN,15,88,0.170455
4653,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,1,0,2,1,2,0,0,2.00,misunderstanding,NaN,29,136,0.213235
4654,tweet4655,Eramos mutuals @LOUISCARAJO,0,3,0,0,0,0,0,NaN,NaN,NaN,3,27,0.111111
4655,tweet4656,Quero desaparecer,0,3,0,0,0,0,0,NaN,NaN,NaN,2,17,0.117647
4656,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...",0,3,0,0,0,0,0,NaN,NaN,NaN,22,112,0.196429
4657,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,1,1,1,1,2,0,0,2.25,NaN,NaN,7,41,0.170732
4658,tweet4659,1-1 Quién desempata? 🤔,0,3,0,0,0,0,0,NaN,NaN,NaN,4,22,0.181818
4659,tweet4660,Siento q hace una semana se fue y recién van t...,0,3,0,0,0,0,0,NaN,NaN,NaN,20,73,0.273973


---
---

# Word Emoji Context Matrix. 🤣 😆 😂 

Descripción: Extracción/guardado de Emojis y depuración del Corpus

Inconvenientes detectados con los emojis:

* 1 Resulta que los emojis hemos podido verificar que countvectorizer los ignora. Es decir si las frases fueran solo emojis el shape de la matriz word context sería (1,1), con valor empty.

* 2 Aun suponiendo que exista alguna alternativa a countvectorizer muchas personas escriben palabras juntas con emojis, e.g.: 'man👨', lo que podría generar un token más del vocabulario que innecesario. Además pueden escribir varios emojis juntos 👨, 👨👨, 👨👨👨, ... generando también un problema en su identificación.

La propuesta ha sido realizar un preproceso como el siguiente. Además, para solucionar el problema de los emoticonos, en lugar de pasárselos a countvectorizer en bruto utilizamos la decodificación a texto de la librería emoji: **emoji.demojize()**

In [101]:
# %%capture
!pip install emoji

In [102]:
import emoji

In [103]:
def extract_emojis(tweet):
  return ''.join(c for c in tweet if c in emoji.UNICODE_EMOJI_ENGLISH)

def get_emojilist(tweet):
  emojistring = extract_emojis(tweet)
  emojilist = [emoji for emoji in emojistring]
  for emoji in emojilist:
    tweet = tweet.replace(emoji, "")
  return tweet, emojilist

def get_emoji_sentence(tweet):
  tweet, emojilist = get_emojilist(tweet)
  emoji_sentence = emoji.demojize(' '.join(emojilist))
  return tweet, emoji_sentence

def filter(text_df):
  filtered_df = pd.DataFrame(map(get_emoji_sentence, text_df['text'].to_list()))
  filtered_df.rename(columns={0: 'sentences_without_emojis', 1: 'emoji_sentences'}, inplace=True)
  return filtered_df

df_filter_output = filter(text)

df_cleaned_sentences = df_filter_output[['sentences_without_emojis']]
# df_cleaned_sentences = df_filter_output['emoji_sentences']

text['text_v2'] = df_filter_output['sentences_without_emojis'].to_list()

emoji_sentences = df_filter_output['emoji_sentences']

text['text_emojis'] = emoji_sentences.to_list()


def get_emoji_vocab(emoji_sentence):
  analyzer = CountVectorizer(binary=False, analyzer='word', # stop_words=more_stop_words,
                              ngram_range=(1, 1)).build_analyzer()
  return (emoji for emoji in analyzer(emoji_sentence))

In [104]:
from sklearn.feature_extraction.text import  CountVectorizer
cv_emoji = CountVectorizer(analyzer=get_emoji_vocab)

In [105]:
try:
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
# Just if there is no emoji
except ValueError:
  emoji_sentences.loc[0,0] = emoji.demojize("🆘")
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
  """emoji_sentences_test = emoji_sentences"""
  """emoji_sentences_test.loc[0,0] = "🆘🆘🆘🆘🆘🆘🆘🆘"""
  """text['text_emojis'] = emoji_sentences_test.to_list()"""
# cv_emoji.get_feature_names()

In [106]:
cv_emoji.get_feature_names()

['grinning_squinting_face', 'thinking_face']

In [107]:
text

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target,len_token,n_char,ratio,text_v2,text_emojis
4650,tweet4651,“Te voy a dejar encerrado todo el día y sin In...,1,0,2,0,2,1,0,2.40,NaN,NaN,30,139,0.215827,“Te voy a dejar encerrado todo el día y sin In...,
4651,tweet4652,Es verdad eso que dicen que después de los qui...,0,3,0,0,0,0,0,NaN,NaN,NaN,13,73,0.178082,Es verdad eso que dicen que después de los qui...,:grinning_squinting_face: :grinning_squinting_...
4652,tweet4653,En mi defensa diré que fui educado mientras El...,0,3,0,0,0,0,0,NaN,NaN,NaN,15,88,0.170455,En mi defensa diré que fui educado mientras El...,
4653,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,1,0,2,1,2,0,0,2.00,misunderstanding,NaN,29,136,0.213235,Ahorita en la calle escuche a dos vatos decir:...,
4654,tweet4655,Eramos mutuals @LOUISCARAJO,0,3,0,0,0,0,0,NaN,NaN,NaN,3,27,0.111111,Eramos mutuals @LOUISCARAJO,
4655,tweet4656,Quero desaparecer,0,3,0,0,0,0,0,NaN,NaN,NaN,2,17,0.117647,Quero desaparecer,
4656,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...",0,3,0,0,0,0,0,NaN,NaN,NaN,22,112,0.196429,"Le saque platica a una de ellas, y pues ehhh.\...",
4657,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,1,1,1,1,2,0,0,2.25,NaN,NaN,7,41,0.170732,Lo que veo en matemáticas: 我不明白這我不明白這. :|,
4658,tweet4659,1-1 Quién desempata? 🤔,0,3,0,0,0,0,0,NaN,NaN,NaN,4,22,0.181818,1-1 Quién desempata?,:thinking_face:
4659,tweet4660,Siento q hace una semana se fue y recién van t...,0,3,0,0,0,0,0,NaN,NaN,NaN,20,73,0.273973,Siento q hace una semana se fue y recién van t...,


---
---


# Detector de cansinos y corrección de repeticiones innecesarias

En twitter hay mucho "pesao" con el pulgar "cansao" y lo dejan mucho tiempo en el móooooooooovil pulsando una letraaaaa. A todos estos, los consideramos unos pesaos y lo vamos a tener en cuenta. No quieren trabajar.

¿Por qué corregiremos también a los pesaos? Porque spellchecker no puede, es fácil ver esto con un ejemplo en código. 

In [108]:
def is_cansino(tweet, tolerancia:int=2):
  # Consideramos pesados a todos aquellos
  # que repitan 3 veces una letra
  """
  Esta función identifica a los pesaos
  Args:
    tolerancia (int): tolerancia al pesao
  """
  l1 = ""
  rep = 0
  letra = ""
  tweet_aux = tweet + '<eos>'
  for index, letra in enumerate(tweet_aux):
      if letra == l1:
          rep += 1
      elif rep >= tolerancia:
          return l1, rep, tolerancia, 1*True
      else:
        rep = 0
        l1 = letra
  return letra, 0, tolerancia, 1*False

def get_cansinos(corpus):
  L = list(map(is_cansino, corpus))
  return [ret[3] for ret in L]

###################################Ç
##### Correccion repeticiones #####
###################################

# Esto es beta solo corrige la primera
# tanda de repeticiones se puede mejorar

def tweet_corrector(tweet):
    letra, rep, tolerancia, bool_ = is_cansino(tweet)
    if bool_ == 1:
        tweet = tweet.replace(rep*letra, '')
        tweet = tweet_corrector(tweet)
    return tweet

def corpus_corrector(corpus):
    L = list(map(tweet_corrector, corpus))
    return L

In [109]:
"""
text.loc[7,'text_v2'] = 'Meeee aaaaabuuuuurroooooo'
text
"""

"\ntext.loc[7,'text_v2'] = 'Meeee aaaaabuuuuurroooooo'\ntext\n"

In [110]:
text['detected_text_cansinos'] = get_cansinos(text['text_v2'].to_list())
text['detected_emojis_cansinos'] = get_cansinos(text['text_emojis'].to_list())
text['detected_cansinos'] = (text['detected_text_cansinos'] + text['detected_emojis_cansinos']) % 2
# text['detected_emojis_cansinos] | text['detected_text_cansinos'] 

text['text_v3'] = corpus_corrector(text['text_v2'].to_list())

text

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target,len_token,n_char,ratio,text_v2,text_emojis,detected_text_cansinos,detected_emojis_cansinos,detected_cansinos,text_v3
4650,tweet4651,“Te voy a dejar encerrado todo el día y sin In...,1,0,2,0,2,1,0,2.40,NaN,NaN,30,139,0.215827,“Te voy a dejar encerrado todo el día y sin In...,,0,0,0,“Te voy a dejar encerrado todo el día y sin In...
4651,tweet4652,Es verdad eso que dicen que después de los qui...,0,3,0,0,0,0,0,NaN,NaN,NaN,13,73,0.178082,Es verdad eso que dicen que después de los qui...,:grinning_squinting_face: :grinning_squinting_...,0,0,0,Es verdad eso que dicen que después de los qui...
4652,tweet4653,En mi defensa diré que fui educado mientras El...,0,3,0,0,0,0,0,NaN,NaN,NaN,15,88,0.170455,En mi defensa diré que fui educado mientras El...,,0,0,0,En mi defensa diré que fui educado mientras El...
4653,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,1,0,2,1,2,0,0,2.00,misunderstanding,NaN,29,136,0.213235,Ahorita en la calle escuche a dos vatos decir:...,,0,0,0,Ahorita en la calle escuche a dos vatos decir:...
4654,tweet4655,Eramos mutuals @LOUISCARAJO,0,3,0,0,0,0,0,NaN,NaN,NaN,3,27,0.111111,Eramos mutuals @LOUISCARAJO,,0,0,0,Eramos mutuals @LOUISCARAJO
4655,tweet4656,Quero desaparecer,0,3,0,0,0,0,0,NaN,NaN,NaN,2,17,0.117647,Quero desaparecer,,0,0,0,Quero desaparecer
4656,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...",0,3,0,0,0,0,0,NaN,NaN,NaN,22,112,0.196429,"Le saque platica a una de ellas, y pues ehhh.\...",,1,0,1,"Le saque platica a una de ellas, y pues eh.\ns..."
4657,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,1,1,1,1,2,0,0,2.25,NaN,NaN,7,41,0.170732,Lo que veo en matemáticas: 我不明白這我不明白這. :|,,0,0,0,Lo que veo en matemáticas: 我不明白這我不明白這. :|
4658,tweet4659,1-1 Quién desempata? 🤔,0,3,0,0,0,0,0,NaN,NaN,NaN,4,22,0.181818,1-1 Quién desempata?,:thinking_face:,0,0,0,1-1 Quién desempata?
4659,tweet4660,Siento q hace una semana se fue y recién van t...,0,3,0,0,0,0,0,NaN,NaN,NaN,20,73,0.273973,Siento q hace una semana se fue y recién van t...,,1,0,1,Siento q hace una semana se fue y recién van t...


---
---

# SpellChecker (Corrector ortográfico)

In [111]:
# SpellChecker creo que no hace falta, pero por si acaso la dejo
# !pip install SpellChecker

In [112]:
# pyspellchecker si hace falta instalarla
# %%capture
!pip install pyspellchecker
from spellchecker import SpellChecker

In [113]:
# Con distance=1 el corrector es más conservador 
spell_check = SpellChecker(language='es', distance=1) # Gracias Yoan
# spell_check.correction("-")
# (9spell_check  = SpellChecker(language='es', distance=100)
# spell_check.correction("chicxs cómo hestais")

In [114]:
string = 'hola'
'a' in string

True

In [115]:
def corrector_ortografico(tweet):
  tweet = tweet.replace('?', ' ? ')
  tweet = tweet.replace('¿', ' ¿ ')
  tweet = tweet.replace('...', ' . ')
  tweet = tweet.replace('.', ' . ')
  tweet = tweet.replace('—', ' . ')
  tweet = tweet.replace('. .', ' . ')
  tweet = tweet.replace('.  .', ' . ')
  tweet = tweet.replace('.   .', ' . ')
  
  tweet = tweet.replace('\n', '').split(' ')
  

  corrected_tweet = [spell_check.correction(token) if token not in ['', '-', '?', '¿', '...', '.'] else token for token in tweet]

  return ' '.join(corrected_tweet)

def corrector_corpus(corpus):
  return list(map(corrector_ortografico, corpus))

In [116]:
text['text_v4'] = corrector_corpus(text['text_v3'])

In [117]:
text[['text_v3', 'text_v4']]

,text_v3,text_v4
4650,“Te voy a dejar encerrado todo el día y sin In...,te voy a dejar encerrado todo el día y sin Int...
4651,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...
4652,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...
4653,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir ...
4654,Eramos mutuals @LOUISCARAJO,Eramos mutuas @LOUISCARAJO
4655,Quero desaparecer,Quero desaparecer
4656,"Le saque platica a una de ellas, y pues eh.\ns...",Le saque platica a una de ellas y pues eh . si...
4657,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|
4658,1-1 Quién desempata?,1-1 Quién desempata ?
4659,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...


In [118]:
text['text_v4'].iloc[6]

'Le saque platica a una de ellas y pues eh . sigo siendo el rechazado del colegio . me deberia de pegar un tiro . '

---
---

# Obtención de Stop Words

In [119]:
# %%capture
!pip install stop_words

In [120]:
from stop_words import get_stop_words # Se añade librería para obtener las stop_word de cualquier idioma
!wget "https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt"

# Se obtienen las stop_words en español
import pickle
with open("more_stop_words.txt", "rb") as f:
  list_test = pickle.load(f)

stop_words = get_stop_words("spanish")
more_stop_words = stop_words + list_test
more_stop_words.sort()

--2021-05-23 15:28:35--  https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10258 (10K) [application/octet-stream]
Saving to: ‘more_stop_words.txt.2’

more_stop_words.txt 100%[===================>]  10.02K  --.-KB/s    in 0s      

2021-05-23 15:28:35 (86.7 MB/s) - ‘more_stop_words.txt.2’ saved [10258/10258]



# ELIMINACIÓN DE IDIOMAS ALEJADOS DEL ESPAÑOL

In [121]:
!pip install spacy_langdetect

In [122]:
# frase de prueba
# text = '¿Hola cómo سمبأسكمق estáis? 종현아생일축하해'

In [123]:
import spacy
from spacy_langdetect import LanguageDetector

# sentence level language detection
dict_ = {}

nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

# Eliminamos palabras koreanas, vietnamitas, japonesas o árabes
L = ['ko', 'vi', 'ja', 'ar', 'zh']

print("v3: ", text['text_v3'].iloc[6])
print("\n\n v4: ", text['text_v4'].iloc[6])

df_cleaned_sentences_list = df_cleaned_sentences['sentences_without_emojis'].to_list()

df_cleaned_sentences_list = text['text_v4'].to_list()

for index, sentence in enumerate(df_cleaned_sentences_list):
  doc = nlp(sentence)
  for subsent in doc.sents:
    
    if subsent._.language['language'] in L:
      
      df_cleaned_sentences_list[index] = sentence.replace(subsent.text,'')
df_cleaned_sentences_list

text['text_v5'] = df_cleaned_sentences_list


text

v3:  Le saque platica a una de ellas, y pues eh.
sigo siendo el rechazado del colegio.
me deberia de pegar un tiro.


 v4:  Le saque platica a una de ellas y pues eh . sigo siendo el rechazado del colegio . me deberia de pegar un tiro . 


,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target,len_token,n_char,ratio,text_v2,text_emojis,detected_text_cansinos,detected_emojis_cansinos,detected_cansinos,text_v3,text_v4,text_v5
4650,tweet4651,“Te voy a dejar encerrado todo el día y sin In...,1,0,2,0,2,1,0,2.40,NaN,NaN,30,139,0.215827,“Te voy a dejar encerrado todo el día y sin In...,,0,0,0,“Te voy a dejar encerrado todo el día y sin In...,te voy a dejar encerrado todo el día y sin Int...,te voy a dejar encerrado todo el día y sin Int...
4651,tweet4652,Es verdad eso que dicen que después de los qui...,0,3,0,0,0,0,0,NaN,NaN,NaN,13,73,0.178082,Es verdad eso que dicen que después de los qui...,:grinning_squinting_face: :grinning_squinting_...,0,0,0,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...
4652,tweet4653,En mi defensa diré que fui educado mientras El...,0,3,0,0,0,0,0,NaN,NaN,NaN,15,88,0.170455,En mi defensa diré que fui educado mientras El...,,0,0,0,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...
4653,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,1,0,2,1,2,0,0,2.00,misunderstanding,NaN,29,136,0.213235,Ahorita en la calle escuche a dos vatos decir:...,,0,0,0,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir ...,Ahorita en la calle escuche a dos vatos decir ...
4654,tweet4655,Eramos mutuals @LOUISCARAJO,0,3,0,0,0,0,0,NaN,NaN,NaN,3,27,0.111111,Eramos mutuals @LOUISCARAJO,,0,0,0,Eramos mutuals @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,Eramos mutuas @LOUISCARAJO
4655,tweet4656,Quero desaparecer,0,3,0,0,0,0,0,NaN,NaN,NaN,2,17,0.117647,Quero desaparecer,,0,0,0,Quero desaparecer,Quero desaparecer,Quero desaparecer
4656,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...",0,3,0,0,0,0,0,NaN,NaN,NaN,22,112,0.196429,"Le saque platica a una de ellas, y pues ehhh.\...",,1,0,1,"Le saque platica a una de ellas, y pues eh.\ns...",Le saque platica a una de ellas y pues eh . si...,Le saque platica a una de ellas y pues eh . si...
4657,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,1,1,1,1,2,0,0,2.25,NaN,NaN,7,41,0.170732,Lo que veo en matemáticas: 我不明白這我不明白這. :|,,0,0,0,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|
4658,tweet4659,1-1 Quién desempata? 🤔,0,3,0,0,0,0,0,NaN,NaN,NaN,4,22,0.181818,1-1 Quién desempata?,:thinking_face:,0,0,0,1-1 Quién desempata?,1-1 Quién desempata ?,1-1 Quién desempata ?
4659,tweet4660,Siento q hace una semana se fue y recién van t...,0,3,0,0,0,0,0,NaN,NaN,NaN,20,73,0.273973,Siento q hace una semana se fue y recién van t...,,1,0,1,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...


---
---

# CREACION MATRIZ WORD CONTEXT DEL TEXTO DEPURADO (SIN EMOJIS y sin idiomas poco habituales)

In [124]:
from nltk.stem.snowball import SpanishStemmer
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=more_stop_words,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))
    
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(analyzer=spanish_stemmer, stop_words=more_stop_words, lowercase=True)
# tf_idf = TfidfTransformer(smooth_idf=True, use_idf=True)
c_vec_mat = c_vec.fit_transform(text['text_v5'])
# tf_idf_mat = tf_idf.fit_transform(c_vec_mat)
c_vec_mat.shape

(10, 64)

In [125]:
df_c_vec = pd.DataFrame(c_vec_mat.toarray(), columns=c_vec.get_feature_names())
print(df_c_vec)

   ahorit  amig  andre  bordill  call  ...  ver  verd  viol  voy  我不明白這我不明白這
0       0     0      0        0     0  ...    1     0     0    1           0
1       0     1      0        0     0  ...    0     1     0    0           0
2       0     0      0        1     0  ...    0     0     0    0           0
3       1     0      2        0     1  ...    0     0     1    1           0
4       0     0      0        0     0  ...    0     0     0    0           0
5       0     0      0        0     0  ...    0     0     0    0           0
6       0     0      0        0     0  ...    0     0     0    0           0
7       0     0      0        0     0  ...    0     0     0    0           1
8       0     0      0        0     0  ...    0     0     0    0           0
9       0     0      0        0     0  ...    0     0     0    0           0

[10 rows x 64 columns]


---

## Uso de Spacy para identificar punchlines

In [126]:
!python -m spacy download es_core_news_md

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [127]:
import es_core_news_md
import numpy as np

In [128]:
nlp = es_core_news_md.load()

In [129]:
noun_ispresent = np.zeros(len(text['text_v5']))
adj_ispresent = np.zeros(len(text['text_v5']))
verb_ispresent = np.zeros(len(text['text_v5']))

In [130]:
general_cats = []
for idx, sample in enumerate(text['text_v5'].to_list()):
  doc_analysis = nlp(sample)
  categories_sequence = ""
  for token in doc_analysis:
    if token.pos_ == "NOUN":
      noun_ispresent[idx] = 1
    if token.pos_ == "ADJ":
      adj_ispresent[idx] = 1
    if token.pos_ == "VERB":
      verb_ispresent[idx] = 1
    categories_sequence +=  token.pos_ + " "
  general_cats.append(categories_sequence)


In [131]:
text['PoSTags'] = general_cats

In [132]:
text['HasNoun'] = noun_ispresent
text['HasAdj'] = adj_ispresent
text['HasVerb'] = verb_ispresent

In [133]:
text

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target,len_token,n_char,ratio,text_v2,text_emojis,detected_text_cansinos,detected_emojis_cansinos,detected_cansinos,text_v3,text_v4,text_v5,PoSTags,HasNoun,HasAdj,HasVerb
4650,tweet4651,“Te voy a dejar encerrado todo el día y sin In...,1,0,2,0,2,1,0,2.40,NaN,NaN,30,139,0.215827,“Te voy a dejar encerrado todo el día y sin In...,,0,0,0,“Te voy a dejar encerrado todo el día y sin In...,te voy a dejar encerrado todo el día y sin Int...,te voy a dejar encerrado todo el día y sin Int...,PRON AUX ADP AUX ADJ DET DET NOUN CONJ ADP PRO...,1.0,1.0,1.0
4651,tweet4652,Es verdad eso que dicen que después de los qui...,0,3,0,0,0,0,0,NaN,NaN,NaN,13,73,0.178082,Es verdad eso que dicen que después de los qui...,:grinning_squinting_face: :grinning_squinting_...,0,0,0,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,AUX NOUN PRON SCONJ VERB SCONJ ADV ADP DET NUM...,1.0,0.0,1.0
4652,tweet4653,En mi defensa diré que fui educado mientras El...,0,3,0,0,0,0,0,NaN,NaN,NaN,15,88,0.170455,En mi defensa diré que fui educado mientras El...,,0,0,0,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,ADP DET NOUN VERB SCONJ AUX VERB SCONJ PROPN P...,1.0,0.0,1.0
4653,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,1,0,2,1,2,0,0,2.00,misunderstanding,NaN,29,136,0.213235,Ahorita en la calle escuche a dos vatos decir:...,,0,0,0,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir ...,Ahorita en la calle escuche a dos vatos decir ...,ADV ADP DET NOUN VERB ADP NUM NOUN VERB PUNCT ...,1.0,0.0,1.0
4654,tweet4655,Eramos mutuals @LOUISCARAJO,0,3,0,0,0,0,0,NaN,NaN,NaN,3,27,0.111111,Eramos mutuals @LOUISCARAJO,,0,0,0,Eramos mutuals @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,VERB NOUN ADJ,1.0,1.0,1.0
4655,tweet4656,Quero desaparecer,0,3,0,0,0,0,0,NaN,NaN,NaN,2,17,0.117647,Quero desaparecer,,0,0,0,Quero desaparecer,Quero desaparecer,Quero desaparecer,VERB VERB,0.0,0.0,1.0
4656,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...",0,3,0,0,0,0,0,NaN,NaN,NaN,22,112,0.196429,"Le saque platica a una de ellas, y pues ehhh.\...",,1,0,1,"Le saque platica a una de ellas, y pues eh.\ns...",Le saque platica a una de ellas y pues eh . si...,Le saque platica a una de ellas y pues eh . si...,PRON VERB VERB ADP PRON ADP PRON CONJ SCONJ PR...,1.0,1.0,1.0
4657,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,1,1,1,1,2,0,0,2.25,NaN,NaN,7,41,0.170732,Lo que veo en matemáticas: 我不明白這我不明白這. :|,,0,0,0,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,DET PRON VERB ADP NOUN PROPN PUNCT SPACE PROPN,1.0,0.0,1.0
4658,tweet4659,1-1 Quién desempata? 🤔,0,3,0,0,0,0,0,NaN,NaN,NaN,4,22,0.181818,1-1 Quién desempata?,:thinking_face:,0,0,0,1-1 Quién desempata?,1-1 Quién desempata ?,1-1 Quién desempata ?,NUM PUNCT NUM PRON VERB PUNCT SPACE,0.0,0.0,1.0
4659,tweet4660,Siento q hace una semana se fue y recién van t...,0,3,0,0,0,0,0,NaN,NaN,NaN,20,73,0.273973,Siento q hace una semana se fue y recién van t...,,1,0,1,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,VERB CONJ AUX DET NOUN PRON AUX CONJ ADV VERB ...,1.0,0.0,1.0


In [134]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.3MB 2.8MB/s 
     |████████████████████████████████| 3.3MB 18.3MB/s 
     |████████████████████████████████| 901kB 38.6MB/s 
     |████████████████████████████████| 1.2MB 2.8MB/s 


In [135]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification, pipeline

In [136]:
# Autotokenizador
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")

# Elección/Descarga del modelo Preentrenado
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

In [137]:
def get_corpus_translation(corpus):
  from transformers import pipeline
  pline = pipeline("translation_es_to_en", model=model, tokenizer=tokenizer)

  def get_tweet_translation(tweet):

    tweet = tweet.lower()

    tweet = tweet.replace('.', ' <.> ')
    tweet = tweet.replace('...', ' . ')
    tweet = tweet.replace(' q ', ' que ')
    tweet = tweet.replace(' k ', ' que ')
    tweet = tweet.replace(' d ', ' de ')

    return pline(tweet)[0]['translation_text'] # punto <point>

  return list(map(get_tweet_translation, corpus['text_v5'].to_list()))

text['en_text'] = get_corpus_translation(text)

def recover_point(tweet):
  return tweet.replace('<.>', ' . ')

def recover_point_corpus(corpus):
  return list(map(recover_point, corpus))

text['en_text'] = recover_point_corpus(text['en_text'])



In [138]:
text = text[['id', 'text', 'text_v2', 'text_v3', 'text_v4', 'text_v5',
             'en_text', 'len_token', 'n_char', 'ratio', 'detected_cansinos',
             'detected_emojis_cansinos', 'detected_text_cansinos', 'text_emojis', 'HasNoun', 'HasAdj', 'HasVerb', 'PoSTags', 'is_humor',
             'votes_no', 'votes_1', 'votes_2', 'votes_3','votes_4', 'votes_5', 'humor_rating', 'humor_mechanism',
             'humor_target']]

text.replace({'text_emojis': {'': 'no_emojis'}}, inplace= True)

corpus = text.copy()

In [ ]:
#corpus['text_emojis'][3]

In [139]:
corpus.to_csv('cleaned_corpus.csv', index=False, sep = ',')

In [140]:
corpus['text_emojis'].loc[4658]

':thinking_face:'

In [141]:
test_corpus = pd.read_csv('cleaned_corpus.csv', header=0, sep=',')

test_corpus.tail(8)

,id,text,text_v2,text_v3,text_v4,text_v5,en_text,len_token,n_char,ratio,detected_cansinos,detected_emojis_cansinos,detected_text_cansinos,text_emojis,HasNoun,HasAdj,HasVerb,PoSTags,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
2,tweet4653,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,in my defense I will say that I was educated w...,15,88,0.170455,0,0,0,no_emojis,1.0,0.0,1.0,ADP DET NOUN VERB SCONJ AUX VERB SCONJ PROPN P...,0,3,0,0,0,0,0,NaN,NaN,NaN
3,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir ...,Ahorita en la calle escuche a dos vatos decir ...,right now on the street listen to two vatos sa...,29,136,0.213235,0,0,0,no_emojis,1.0,0.0,1.0,ADV ADP DET NOUN VERB ADP NUM NOUN VERB PUNCT ...,1,0,2,1,2,0,0,2.00,misunderstanding,NaN
4,tweet4655,Eramos mutuals @LOUISCARAJO,Eramos mutuals @LOUISCARAJO,Eramos mutuals @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,we were mutual @louiscarajo,3,27,0.111111,0,0,0,no_emojis,1.0,1.0,1.0,VERB NOUN ADJ,0,3,0,0,0,0,0,NaN,NaN,NaN
5,tweet4656,Quero desaparecer,Quero desaparecer,Quero desaparecer,Quero desaparecer,Quero desaparecer,I want to disappear.,2,17,0.117647,0,0,0,no_emojis,0.0,0.0,1.0,VERB VERB,0,3,0,0,0,0,0,NaN,NaN,NaN
6,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...","Le saque platica a una de ellas, y pues ehhh.\...","Le saque platica a una de ellas, y pues eh.\ns...",Le saque platica a una de ellas y pues eh . si...,Le saque platica a una de ellas y pues eh . si...,"I take one of them out of talk, and then I'm s...",22,112,0.196429,1,0,1,no_emojis,1.0,1.0,1.0,PRON VERB VERB ADP PRON ADP PRON CONJ SCONJ PR...,0,3,0,0,0,0,0,NaN,NaN,NaN
7,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,what I see in mathematics <..> :,7,41,0.170732,0,0,0,no_emojis,1.0,0.0,1.0,DET PRON VERB ADP NOUN PROPN PUNCT SPACE PROPN,1,1,1,1,2,0,0,2.25,NaN,NaN
8,tweet4659,1-1 Quién desempata? 🤔,1-1 Quién desempata?,1-1 Quién desempata?,1-1 Quién desempata ?,1-1 Quién desempata ?,1-1 who unpacks?,4,22,0.181818,0,0,0,:thinking_face:,0.0,0.0,1.0,NUM PUNCT NUM PRON VERB PUNCT SPACE,0,3,0,0,0,0,0,NaN,NaN,NaN
9,tweet4660,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,I feel like a week ago he left and it's only t...,20,73,0.273973,1,0,1,no_emojis,1.0,0.0,1.0,VERB CONJ AUX DET NOUN PRON AUX CONJ ADV VERB ...,0,3,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
test_corpus['text_emojis'][~test_corpus['text_emojis'].isna()]

0                                            no_emojis
1    :grinning_squinting_face: :grinning_squinting_...
2                                            no_emojis
3                                            no_emojis
4                                            no_emojis
5                                            no_emojis
6                                            no_emojis
7                                            no_emojis
8                                      :thinking_face:
9                                            no_emojis
Name: text_emojis, dtype: object

In [ ]:
test_corpus.iloc[6]['text']

'Le saque platica a una de ellas, y pues ehhh.\nsigo siendo el rechazado del colegio.\nme deberia de pegar un tiro.'

In [ ]:
test_corpus.iloc[6]['text_v3']

'Le saque platica a una de ellas, y pues eh.\nsigo siendo el rechazado del colegio.\nme deberia de pegar un tiro.'

In [ ]:
test_corpus.iloc[6]['text_v4']

'Le saque platica a una de ellas y pues eh . sigo siendo el rechazado del colegio . me deberia de pegar un tiro . '

In [ ]:
test_corpus.iloc[6]['text_v5']

'Le saque platica a una de ellas y pues eh . sigo siendo el rechazado del colegio . me deberia de pegar un tiro . '

In [ ]:
test_corpus.iloc[6]['en_text']

"I take one of them out of talk, and then I'm still the rejected one from school. I should have shot myself."

**I'm bored** antes se traducía como **meeee bored**, así que hemos ganado en calidad.

Generar a partir de text_v2, text_v3 donde se pase text_v2 y se haga el spell_checker

In [ ]:
# eliminar nulos

df_train = test_corpus[~test_corpus.text_v5.isna()]
df_train

,id,text,text_v2,text_v3,text_v4,text_v5,en_text,len_token,n_char,ratio,detected_cansinos,detected_emojis_cansinos,detected_text_cansinos,text_emojis,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
0,tweet4651,“Te voy a dejar encerrado todo el día y sin In...,“Te voy a dejar encerrado todo el día y sin In...,“Te voy a dejar encerrado todo el día y sin In...,te voy a dejar encerrado todo el día y sin Int...,te voy a dejar encerrado todo el día y sin Int...,I'm going to leave you locked up all day and w...,30,139,0.215827,0,0,0,no_emojis,1,0,2,0,2,1,0,2.40,NaN,NaN
1,tweet4652,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,Es verdad eso que dicen que después de los qui...,It is true that they say that after the fiftee...,13,73,0.178082,0,0,0,:grinning_squinting_face: :grinning_squinting_...,0,3,0,0,0,0,0,NaN,NaN,NaN
2,tweet4653,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,En mi defensa diré que fui educado mientras El...,in my defense I will say that I was educated w...,15,88,0.170455,0,0,0,no_emojis,0,3,0,0,0,0,0,NaN,NaN,NaN
3,tweet4654,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir:...,Ahorita en la calle escuche a dos vatos decir ...,Ahorita en la calle escuche a dos vatos decir ...,right now on the street listen to two vatos sa...,29,136,0.213235,0,0,0,no_emojis,1,0,2,1,2,0,0,2.00,misunderstanding,NaN
4,tweet4655,Eramos mutuals @LOUISCARAJO,Eramos mutuals @LOUISCARAJO,Eramos mutuals @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,Eramos mutuas @LOUISCARAJO,we were mutual @louiscarajo,3,27,0.111111,0,0,0,no_emojis,0,3,0,0,0,0,0,NaN,NaN,NaN
5,tweet4656,Quero desaparecer,Quero desaparecer,Quero desaparecer,Quero desaparecer,Quero desaparecer,I want to disappear.,2,17,0.117647,0,0,0,no_emojis,0,3,0,0,0,0,0,NaN,NaN,NaN
6,tweet4657,"Le saque platica a una de ellas, y pues ehhh.\...","Le saque platica a una de ellas, y pues ehhh.\...","Le saque platica a una de ellas, y pues eh.\ns...",Le saque platica a una de ellas y pues eh . si...,Le saque platica a una de ellas y pues eh . si...,"I take one of them out of talk, and then I'm s...",22,112,0.196429,1,0,1,no_emojis,0,3,0,0,0,0,0,NaN,NaN,NaN
7,tweet4658,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas: 我不明白這我不明白這. :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,Lo que veo en matemáticas 我不明白這我不明白這 . :|,what I see in mathematics <..> :,7,41,0.170732,0,0,0,no_emojis,1,1,1,1,2,0,0,2.25,NaN,NaN
8,tweet4659,1-1 Quién desempata? 🤔,1-1 Quién desempata?,1-1 Quién desempata?,1-1 Quién desempata ?,1-1 Quién desempata ?,1-1 who unpacks?,4,22,0.181818,0,0,0,:thinking_face:,0,3,0,0,0,0,0,NaN,NaN,NaN
9,tweet4660,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,Siento q hace una semana se fue y recién van t...,I feel like a week ago he left and it's only t...,20,73,0.273973,1,0,1,no_emojis,0,3,0,0,0,0,0,NaN,NaN,NaN
